<a href="https://colab.research.google.com/github/aimat163/python/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B8%D1%80%D1%83%D0%B5%D0%BC_%D0%BF%D0%BE%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations

In [ ]:
df2 = pd.read_csv('/content/itogovui2.csv')

In [ ]:
def prepare_sessions_and_purchases(df):
    """
    Подготавливает данные, связывая события и покупки в одной сессии

    Сессия = client_id + дата
    Покупка может быть в любой части дня (в отдельной строке)
    События могут быть в других строках (но же день и client_id)
    """

    df = df.copy()

    # Преобразуем дату
    df['дата'] = pd.to_datetime(df['ym:pv:dateTime']).dt.date

    # Создаём уникальный ключ сессии
    df['session_key'] = df['ym:pv:clientID'].astype(str) + '_' + df['дата'].astype(str)

    # Определяем была ли покупка в сессии
    # Покупка есть если в сессии есть хотя бы одна строка где purchase == 1
    purchase_by_session = df.groupby('session_key')['purchase'].max().reset_index()
    purchase_by_session.columns = ['session_key', 'had_purchase']

    # Объединяем с исходными данными
    df = df.merge(purchase_by_session, on='session_key', how='left')

    print("✓ Подготовка данных завершена")
    print(f"  - Всего строк: {len(df):,}")
    print(f"  - Уникальных сессий: {df['session_key'].nunique():,}")
    print(f"  - Сессий с покупками: {(df['had_purchase'] == 1).groupby(df['session_key']).first().sum():,}")

    return df

In [ ]:
df3 = prepare_sessions_and_purchases(df2)

✓ Подготовка данных завершена
  - Всего строк: 44,044
  - Уникальных сессий: 817
  - Сессий с покупками: 711


In [99]:
def analyze_event_sequences(df, max_length=5):

  # Берём только те строки, в которых есть события, сортируем по времени
  df_events = df3[df3['needed_params2'].notna() & (df3['needed_params2'] != '')].copy()
  df_events = df_events.sort_values('ym:pv:dateTime')

  # Для каждой сессии собираем последовательность событий в список
  sequences_data = []
  for session_key, group in df_events.groupby('session_key'):
    # Событие в хронологическом порядке
    events_seq = group['needed_params2'].tolist()
    # Была ли покупка в этой сессии
    purchase = group['had_purchase'].iloc[0]

    sequences_data.append({
        'session_key': session_key,
        'events': events_seq,
        'purchase': purchase
      })

  # Разделяем на купивших и некупивших
  purchaser_sequences = [s['events'] for s in sequences_data if s['purchase'] == 1]
  non_purchaser_sequences = [s['events'] for s in sequences_data if s['purchase'] == 0]

  # Функция группировки событие с подсчетом, например ['A', 'B', 'B'] то будет ['A', 'B (2)'], чтобы не засорять отчет
  def compress_events(seq):
    result = []
    if not seq:
        return result
    count = 1
    prev = seq[0]
    for event in seq[1:]:
        if event == prev:
            count += 1
        else:
            if count > 1:
                result.append(f"{prev} ({count})")
            else:
                result.append(prev)
            prev = event
            count = 1
    if count > 1:
        result.append(f"{prev} ({count})")
    else:
        result.append(prev)
    return result

  compressed_buyer = [compress_events(seq) for seq in purchaser_sequences]
  compressed_non_buyer = [compress_events(seq) for seq in non_purchaser_sequences]

  # Функция для получения всех последовательностей в зависимости от заданной длины
  # Например, из [A, B, C] получаем: (A), (B), (C), (A,B), (B,C), (A,B,C)

  def get_subsequences(seq, max_length=5):
      subseqs = []
      for length in range(1, min(len(seq) + 1, max_length + 1)):
          for i in range(len(seq) - length + 1):
              subseqs.append(tuple(seq[i:i+length]))
      return subseqs

  #Считаем частоту последовательностей у купивших
  purchaser_patterns = Counter()
  for seq in compressed_buyer:
    for subseq in get_subsequences(seq, max_length):
      purchaser_patterns[subseq] += 1

  #Считаем частоту последовательностей у некупивших
  non_purchaser_patterns = Counter()
  for seq in compressed_non_buyer:
    for subseq in get_subsequences(seq, max_length):
      non_purchaser_patterns[subseq] += 1
  #Рассчитываем Lift для каждой подпоследовательности
  results = []
  total_purchasers = len(purchaser_sequences) if len(purchaser_sequences) > 0 else 1
  total_non_purchasers = len(non_purchaser_sequences) if len(non_purchaser_sequences) > 0 else 1

  for pattern, count in purchaser_patterns.most_common(1000):
  #частота события = купившие / всего купивших
    support_purchasers = count / total_purchasers
    support_non_purchasers = non_purchaser_patterns.get(pattern, 0) / total_non_purchasers

  # Lift = вероятность купившие к не купившим
    lift = support_purchasers / support_non_purchasers if support_non_purchasers > 0 else 999

    if count >= 2:  # Минимум 2 сессии для статистики
        results.append({
        'Последовательность': ' → '.join(pattern),
        'Длина': len(pattern),
        'Частота у покупателей': count,
        'Доля покупателей %': round(support_purchasers * 100, 2),
        'Доля не покупателей %': round(support_non_purchasers * 100, 2),
        'Lift': round(lift, 2) if lift != 999 else 0.009
                  })
  return pd.DataFrame(results).sort_values('Lift', ascending=False)

In [100]:
sequences_analysis = analyze_event_sequences(df3, max_length=5)
sequences_analysis

,Последовательность,Длина,Частота у покупателей,Доля покупателей %,Доля не покупателей %,Lift
59,add_to_cart → scroll_pdp (7),2,50,7.52,0.94,7.970
69,scroll_pdp (4) → add_to_cart,2,44,6.62,0.94,7.010
73,add_to_cart → scroll_pdp (5),2,40,6.02,0.94,6.380
77,scroll_pdp → interact_with_photo (6),2,36,5.41,0.94,5.740
78,add_to_cart → scroll_pdp (9),2,36,5.41,0.94,5.740
...,...,...,...,...,...,...
587,interact_with_photo (4) → show_more → viewing_...,3,5,0.75,0.00,0.009
588,scroll_pdp (29),1,5,0.75,0.00,0.009
592,viewing_review_block → scroll_pdp (3) → intera...,3,5,0.75,0.00,0.009
593,show_more (2) → scroll_pdp (6),2,5,0.75,0.00,0.009
